In [14]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from bs4 import BeautifulSoup

data = open('data/Rode 16.67 mi on 08-28-2015.tcx', 'r').read()

data = data.replace('\n', '')
data = data.replace('\t', '')

soup = BeautifulSoup(data)

class Trackpoint(object):
    
    def __init__(self, trackpoint):
        self.trackpoint = trackpoint
        
    def get(self, attr):
        result = self.trackpoint.find(attr)
        if result:
            value = result.find('value')
            if value:
                return value.text
            return result.text
        else:
            return None
            
trackpoints = soup.findAll('trackpoint')

to_dict = []

for trackpoint in trackpoints:
    TP = Trackpoint(trackpoint)
    temp = {}
    for child in trackpoint.children:
        temp[child.name] = TP.get(child.name)
    temp['latitudedegrees'] = TP.get('latitudedegrees')
    temp['longitudedegrees'] = TP.get('longitudedegrees')
    to_dict.append(temp)
    
df = pd.DataFrame.from_dict(to_dict)

df['time'] = pd.to_datetime(df['time'])

df.set_index('time')

df['distancemeters'] = df['distancemeters'].astype(float)
df['altitudemeters'] = df['altitudemeters'].astype(float)
df['latitudedegrees'] = df['latitudedegrees'].astype(float)
df['longitudedegrees'] = df['longitudedegrees'].astype(float)

df.to_excel('data.xlsx')

map_data = df
map_data = map_data.drop(['position'], 1).dropna()

map_data['distancedelta'] = map_data['distancemeters'].diff()

map_data = map_data[map_data.distancedelta > 0]

In [15]:
import folium
from IPython.core.display import HTML


def inline_map(m, width=650, height=500):
    """Takes a folium instance and embed HTML."""
    m._build_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{}" '
                 'style="width: {}px; height: {}px; '
                 'border: none"></iframe>'.format(srcdoc, width, height))
    return embed


width, height = 650, 500
ride = folium.Map(location=[38.95, -77.07], zoom_start=12,
                    tiles='Open Street Map', width=width, height=height)

In [16]:
ride.line(zip(map_data.latitudedegrees, map_data.longitudedegrees), line_color='#FF0000', line_weight=5)

In [17]:
inline_map(ride)